In [ ]:
# Importa la biblioteca pandas 
import pandas as pd
# Importa la biblioteca requests 
import requests

# Configura pandas para que muestre todas las columnas
pd.set_option('display.max_columns', None)

# ENDPOINT para obtener libros de Stephen King
url = "https://api.nytimes.com/svc/books/v3/reviews.json?author=Stephen+King&api-key=YOUR_API_KEY"

# Realiza una solicitud a la URL
response = requests.get(url)
# Comprueba si la respuesta es exitosa
if response.status_code == 200:
    # Convierte la respuesta a JSON 
    json_response = response.json()

    # Define una función para limpiar los datos JSON y convertirlos en un DataFrame 
    def limpiar_json(datos_json):
        # Crea una lista vacía para almacenar los datos 
        libros = []
        # Crea un conjunto vacío para registrar los ISBN únicos y evitar duplicados
        isbn_list = set()

        # Itera sobre cada iterable en la sección 'results' 
        for item in datos_json['results']:
            # Intenta obtener el campo 'isbn13' de los datos del libro
            # Si es una lista, obtiene el primer elemento; si no, usa el valor tal cual
            # Si 'isbn13' no está presente o es None, usa 'Desconocido'
            isbn = item.get('isbn13')
            isbn = isbn[0] if isinstance(isbn, list) and isbn else 'Desconocido'

            # Si el ISBN no está duplicado
            if isbn not in isbn_list:
                # Obtiene el título del libro, el autor y la fecha de publicación del libro
                # Utiliza valores predeterminados en caso de que no estén 
                titulo = item.get('book_title', 'Desconocido')
                autor = item.get('book_author', 'Desconocido')
                fecha = item.get('publication_dt', 'Desconocido')

                # Agrega un diccionario con los datos del libro a la lista de libros
                libros.append({
                    'Título': titulo,
                    'Autor/a': autor,
                    'ID': isbn,
                    'Fecha': fecha,
                })
                # Añade el ISBN al conjunto para evitar futuros duplicados
                isbn_list.add(isbn)

        # Crea un DataFrame de pandas a partir de la lista de datos de los libros
        return pd.DataFrame(libros)

    # Llama a la función 'limpiar_json' 
    # Asigna el DataFrame resultante a una variable 
    df_libros = limpiar_json(json_response)

    # Imprime el DataFrame para ver los resultados
    print(df_libros)

    # Guarda el DataFrame en un archivo CSV 
    # Establece 'index=False' para no incluir la columna de índices en el archivo
    df_libros.to_csv("Libros_de_Stephen_King.csv", index=False)

# Si la solicitud no fue exitosa, imprime un mensaje de error con el código de estado de la petición
else:
    print(f"No ha funcionado, error tipo {response.status_code}")